In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\korean_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'korean']

In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)  
data

,emd_cd,korean
base_date,,
2018-01-31,50110250,7.209306e+06
2018-02-28,50110250,5.579125e+06
2018-03-31,50110250,7.615021e+06
2018-04-30,50110250,8.498349e+06
2018-05-31,50110250,8.916782e+06
...,...,...
2021-02-28,50130620,4.321265e+06
2021-03-31,50130620,4.744309e+06
2021-04-30,50130620,5.236706e+06


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110640  ##각행정동코드

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,korean
base_date,
2018-01-31,9.935979e+06
2018-02-28,8.702439e+06
2018-03-31,9.799380e+06
2018-04-30,9.211209e+06
2018-05-31,9.187952e+06
2018-06-30,8.732963e+06
2018-07-31,9.801707e+06
2018-08-31,9.683261e+06
2018-09-30,8.557417e+06


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'korean': base_date
 2018-01-31    9.935979e+06
 2018-02-28    8.702439e+06
 2018-03-31    9.799380e+06
 2018-04-30    9.211209e+06
 2018-05-31    9.187952e+06
 2018-06-30    8.732963e+06
 2018-07-31    9.801707e+06
 2018-08-31    9.683261e+06
 2018-09-30    8.557417e+06
 2018-10-31    8.952027e+06
 2018-11-30    9.007342e+06
 2018-12-31    9.474570e+06
 2019-01-31    9.920262e+06
 2019-02-28    9.274388e+06
 2019-03-31    1.015689e+07
 2019-04-30    9.432024e+06
 2019-05-31    9.770665e+06
 2019-06-30    8.752466e+06
 2019-07-31    9.549349e+06
 2019-08-31    9.524536e+06
 2019-09-30    8.779416e+06
 2019-10-31    9.778089e+06
 2019-11-30    1.003027e+07
 2019-12-31    1.043675e+07
 2020-01-31    1.032745e+07
 2020-02-29    8.988173e+06
 2020-03-31    8.374226e+06
 2020-04-30    8.182633e+06
 2020-05-31    8.590476e+06
 2020-06-30    8.515325e+06
 2020-07-31    8.636450e+06
 2020-08-31    8.908982e+06
 2020-09-30    8.325132e+06
 2020-10-31    9.119918e+06
 2020-11-30    9.434029e+06

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -4.184335
p-value : 0.000700
Critical Values :
	1%: -3.600983
	5%: -2.935135
	10%: -2.605963


In [8]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=1238.562, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=1235.370, Time=0.08 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=1234.051, Time=0.04 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=1235.932, Time=0.03 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=1236.516, Time=0.16 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=1237.333, Time=0.24 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=1233.223, Time=0.03 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=1235.129, Time=0.08 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=1236.673, Time=0.09 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=1237.950, Time=0.10 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=1239.725, Time=0.18 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=1234.914, Time=0.09 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=1236.832, Time=0.12 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=1238.868, Time=0.27 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=1239.710, Time=0.19 sec
 ARIMA(3,0

ARIMA(order=(1, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [9]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.korean.values, order=(1,0,0))   

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(1, 0)   Log Likelihood                -613.609
Method:                       css-mle   S.D. of innovations         534280.577
Date:                Sat, 11 Sep 2021   AIC                           1233.218
Time:                        02:08:51   BIC                           1238.431
Sample:                             0   HQIC                          1235.129
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       9.183e+06   1.36e+05     67.399      0.000    8.92e+06    9.45e+06
ar.L1.y        0.4044      0.142      2.849      0.004       0.126       0.683
                                    Roots           

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [10]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([9070393.93181224, 9137456.09239051]), array([534280.57683052, 576318.24278186]), array([[ 8023223.24358514, 10117564.62003934],
       [ 8007893.09290465, 10267019.09187637]]))
